In [3]:
import numpy as np
from unfolding_helper import *

#helper function to see if to faces are normal to eachother
is_normal = lambda s1, s2 : sum([abs(c1) != abs(c2) for c1, c2 in zip(s1, s2)]) != 0

def strip_n_wings(face, mesh):
    all_normals = list(mesh.face_normals())
    face_normals = all_normals[face.idx()]
    
    #Every face normal to inputted face is strip
    strip = [f.idx() for f in mesh.faces() if is_normal(face_normals, all_normals[f.idx()])]
    
    #Every face that is not strip must be wing
    wings = [f.idx() for f in mesh.faces() if f.idx() not in strip]
    
    return (strip, wings)

In [4]:
def get_best_snw(mesh):
    
    #helper function to select three different faces which are normal to eachother
    def three_normal(mesh):
        all_normals = list(mesh.face_normals())
        faces = []
        
        for f in mesh.faces():
            
            #if current face is not not normal to every face in faces append current face to faces
            if not sum([not is_normal(all_normals[f.idx()], all_normals[ff.idx()]) for ff in faces]):
                faces.append(f)
            
            #if we found three normal faces return them
            if len(faces) == 3:
                return faces
        
        return faces
    
    #List of (strip, wings) tuples from faces out of three_normal()
    snw_list = [strip_n_wings(f, mesh) for f in three_normal(mesh)]
    
    #Rating strips based on size by sorting them
    snw_list.sort(key=lambda snw : len(snw[0]))
    
    return snw_list[-1]
    

In [5]:
#start here: modify code to deal with different kinds of errors:
#    what happens if child array is empty but strip isn't?
#    what happens when the unfolded strip overlaps somewhere?
#    --> put every face which causes an error in sperate strip and call strip_2_tree on that strip
#    --> create multiple trees by this, return type is list of trees

#Tipp: you can remove child by id: tree.rem_child(child_id), or by reference tree.rem_child_node(child_node)
#      tree.unfold(mesh) to unfold the mesh according to the spanning tree
#      is_unfolding_overlapping(tree.unfold(mesh)) for the overlap check
def strip_2_tree(mesh, strip):
    #Create Rootnode for spanning tree out of first face in strip
    root = node(strip.pop(0))
    curr = root
    while strip:
        
        #find all adjacent_faces to current face curr                                                                                               
        children = [f for f in strip if f in mesh.get_adjacent_faces_idx(curr.val)]
        
        #choose a child out of possible children, in this case just the first one possible
        child_id = children[0]                                                                                                    
        
        #remove that child from strip and add it to the tree
        strip.remove(child_id)                                                                                                                      
        child_node = node(child_id)                                                                                                                 
        curr.add_child_node(child_node)
        
        #set curr to the freshly added node
        curr = child_node
    
    return root 

In [6]:
#start here: modify this code to deal with a list of trees
#    deal with overlaps
#
#Tipp: you can remove child by id: tree.rem_child(child_id), or by reference tree.rem_child_node(child_node)
#      tree.unfold(mesh) to unfold the mesh according to the spanning tree
#      is_unfolding_overlapping(tree.unfold(mesh)) for the overlap check
def attach_wings(mesh, tree, wings):                                                                                                               
                                                                                                                            
    def attach_wing(tree, wing):
        #Check every adjacent face to the wing if it could be a parent
        for parent_face in mesh.get_adjacent_faces_idx(wing):         
            
            #if it is: insert it and return
            if tree.has(parent_face):                                                                                                               
                tree.insert_child(parent_face, node(wing))                                                                                          
                return
                        
    #attach all wings
    for wing in wings:                                                                                                                              
        attach_wing(tree, wing)                                                                                                                                                                                                                                                                                                                         
    return tree

In [11]:
mesh = Mesh('Models/L2.obj')
strip, wings = get_best_snw(mesh)
tree = strip_2_tree(mesh, strip)
tree = attach_wings(mesh, tree, wings)
unfolding = tree.unfold(mesh)

print(is_unfolding_overlapping(unfolding))
draw_svg(unfolding, 'Unfolded_SVGs/L2.svg')

True


  Warning! Material file 'Models/L2.mtl' not found!
Warning! Material 'L2_wire_061135006' not defined in material file.
